In [ ]:
!pip install -U transformers datasets accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 77.4 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.0 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
  

# opt1.3b

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load OPT-1.3B Model (FP16, No Quantization)
# -----------------------
model_name = "facebook/opt-1.3b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",              # Uses GPU automatically if available
    torch_dtype=torch.float16       # FP16 to reduce memory usage
)
model.eval()

# -----------------------
# 2. Load Dataset (Math QA - Multiple Choice Math Problems)
# -----------------------
dataset = load_dataset("math_qa")
test_set = dataset['test']

# -----------------------
# 3. Prompt Template for Mathematical Reasoning MCQs
# -----------------------
def generate_prompt(problem, choices):
    prompt = f"Problem: {problem}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print("Running evaluation on MathQA with OPT-1.3B...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['Problem']
    choices = [opt.split(') ')[1] for opt in example['options'].split('\n') if ') ' in opt]

    correct_answer_label = example['correct']
    answer_idx = ord(correct_answer_label) - ord('a')
    correct_answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = correct_answer.lower() in output.lower()

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": f"{correct_answer_label.upper()}. {correct_answer}",
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on MathQA (Mathematical Reasoning Task): {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("opt_mathqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to opt_mathqa_predictions.json")


2025-06-17 15:55:14.262249: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750175714.285485     218 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750175714.292611     218 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


pytorch_model.bin:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.44k [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2985 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4475 [00:00<?, ? examples/s]

Running evaluation on MathQA with OPT-1.3B...



100%|██████████| 2985/2985 [1:08:01<00:00,  1.37s/it]


Final Accuracy on MathQA (Mathematical Reasoning Task): 0.8931

Predictions saved to opt_mathqa_predictions.json


# Falcon 1b

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

model_name = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

dataset = load_dataset("math_qa")
test_set = dataset['test']

def generate_prompt(problem, choices):
    prompt = f"Problem: {problem}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

def get_model_output(prompt, max_new_tokens=64):
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

correct = 0
total = 0
predictions = []

print(f"Running evaluation on MathQA with {model_name}...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['Problem']
    choices = [opt.split(') ')[1] for opt in example['options'].split('\n') if ') ' in opt]
    correct_answer_label = example['correct']
    answer_idx = ord(correct_answer_label) - ord('a')
    correct_answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""
    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)
    is_correct = correct_answer.lower() in output.lower()
    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": f"{correct_answer_label.upper()}. {correct_answer}",
        "correct": is_correct
    })
    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on MathQA: {accuracy:.4f}")

with open("falcon_rw1b_mathqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)


tokenizer_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/2985 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4475 [00:00<?, ? examples/s]

Running evaluation on MathQA with tiiuae/falcon-rw-1b...



100%|██████████| 2985/2985 [1:23:01<00:00,  1.67s/it]


Final Accuracy on MathQA: 0.8007


# GPT-Neo-1.3B

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Pythia-1B Model (FP16, No Quantization)
# -----------------------
model_name = "EleutherAI/gpt-neo-1.3B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# -----------------------
# 2. Load Dataset (Math QA - Multiple Choice Math Problems)
# -----------------------
dataset = load_dataset("math_qa")
test_set = dataset['test']

# -----------------------
# 3. Prompt Template for Mathematical Reasoning MCQs
# -----------------------
def generate_prompt(problem, choices):
    prompt = f"Problem: {problem}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print(f"Running evaluation on MathQA with {model_name}...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['Problem']
    choices = [opt.split(') ')[1] for opt in example['options'].split('\n') if ') ' in opt]

    correct_answer_label = example['correct']
    answer_idx = ord(correct_answer_label) - ord('a')
    correct_answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = correct_answer.lower() in output.lower()

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": f"{correct_answer_label.upper()}. {correct_answer}",
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on MathQA with {model_name}: {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("pythia1b_mathqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to pythia1b_mathqa_predictions.json")


Running evaluation on MathQA with EleutherAI/gpt-neo-1.3B...



100%|██████████| 2985/2985 [1:39:09<00:00,  1.99s/it]


Final Accuracy on MathQA with EleutherAI/gpt-neo-1.3B: 0.8536

Predictions saved to pythia1b_mathqa_predictions.json


# StableLM 2 (1.6B)

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Pythia-1B Model (FP16, No Quantization)
# -----------------------
model_name = "stabilityai/stablelm-2-1_6b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# -----------------------
# 2. Load Dataset (Math QA - Multiple Choice Math Problems)
# -----------------------
dataset = load_dataset("math_qa")
test_set = dataset['test']

# -----------------------
# 3. Prompt Template for Mathematical Reasoning MCQs
# -----------------------
def generate_prompt(problem, choices):
    prompt = f"Problem: {problem}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print(f"Running evaluation on MathQA with {model_name}...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['Problem']
    choices = [opt.split(') ')[1] for opt in example['options'].split('\n') if ') ' in opt]

    correct_answer_label = example['correct']
    answer_idx = ord(correct_answer_label) - ord('a')
    correct_answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = correct_answer.lower() in output.lower()

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": f"{correct_answer_label.upper()}. {correct_answer}",
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on MathQA with {model_name}: {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("pythia1b_mathqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to pythia1b_mathqa_predictions.json")


tokenizer_config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.01M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/917k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/784 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.29G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/121 [00:00<?, ?B/s]

Running evaluation on MathQA with stabilityai/stablelm-2-1_6b...



  0%|          | 7/2985 [00:09<1:07:19,  1.36s/it]Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
  1%|          | 18/2985 [00:22<1:07:00,  1.36s/it]Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
  1%|          | 24/2985 [00:29<1:03:34,  1.29s/it]Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
  1%|          | 27/2985 [00:30<44:53,  1.10it/s]Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
  1%|▏         | 38/2985 [00:43<1:02:40,  1.28s/it]Setting `pad_token_id` to `eos_token_id`:100257 for open-end generation.
Setting `pad_token


Final Accuracy on MathQA with stabilityai/stablelm-2-1_6b: 0.7970

Predictions saved to pythia1b_mathqa_predictions.json


# pythia-1b

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from tqdm import tqdm
import json

# -----------------------
# 1. Load Pythia-1B Model (FP16, No Quantization)
# -----------------------
model_name = "EleutherAI/pythia-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# -----------------------
# 2. Load Dataset (Math QA - Multiple Choice Math Problems)
# -----------------------
dataset = load_dataset("math_qa")
test_set = dataset['test']

# -----------------------
# 3. Prompt Template for Mathematical Reasoning MCQs
# -----------------------
def generate_prompt(problem, choices):
    prompt = f"Problem: {problem}\nChoices:\n"
    for idx, choice in enumerate(choices):
        prompt += f"{chr(65+idx)}. {choice}\n"
    prompt += "Answer:"
    return prompt

# -----------------------
# 4. Generate Model Output
# -----------------------
def get_model_output(prompt, max_new_tokens=64):
    device = model.device
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(prompt):].strip()

# -----------------------
# 5. Evaluation
# -----------------------
correct = 0
total = 0
predictions = []

print(f"Running evaluation on MathQA with {model_name}...\n")

for example in tqdm(test_set, total=len(test_set)):
    question = example['Problem']
    choices = [opt.split(') ')[1] for opt in example['options'].split('\n') if ') ' in opt]

    correct_answer_label = example['correct']
    answer_idx = ord(correct_answer_label) - ord('a')
    correct_answer = choices[answer_idx] if 0 <= answer_idx < len(choices) else ""

    prompt = generate_prompt(question, choices)
    output = get_model_output(prompt)

    is_correct = correct_answer.lower() in output.lower()

    predictions.append({
        "question": question,
        "prompt": prompt,
        "output": output,
        "choices": choices,
        "ground_truth_answer": f"{correct_answer_label.upper()}. {correct_answer}",
        "correct": is_correct
    })

    total += 1
    correct += int(is_correct)

accuracy = correct / total if total > 0 else 0.0
print(f"\nFinal Accuracy on MathQA with {model_name}: {accuracy:.4f}")

# -----------------------
# 6. Save Predictions
# -----------------------
with open("pythia1b_mathqa_predictions.json", "w") as f:
    json.dump(predictions, f, indent=2)

print("\nPredictions saved to pythia1b_mathqa_predictions.json")


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.09G [00:00<?, ?B/s]

Running evaluation on MathQA with EleutherAI/pythia-1b...



100%|██████████| 2985/2985 [41:51<00:00,  1.19it/s]


Final Accuracy on MathQA with EleutherAI/pythia-1b: 0.8191

Predictions saved to pythia1b_mathqa_predictions.json
